In [20]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
from scipy.spatial import Delaunay
import tkinter as tk
import math
import sys

%matplotlib qt

def dist(p1, p2): # Calculates the distance between two points in three-dimensional space.
    return math.sqrt((p1[0]-p2[0]) ** 2 + (p1[1]-p2[1]) ** 2 + (p1[2]-p2[2]) ** 2)+0.00000000001

def set_axes_equal(ax):
    # ACQUIRED FROM INTERNET - makes it so the scaling of the axes is consistent between x, y, and z. Why this is not yet a default part of matplotlib is beyond me.

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])

class potVal: # Class for points on the graph, storing x-y-z coordinate data, the index of its equipotential surface (if any), and its potential value from the points around it.
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        totPot = 0
        for n in range(0, len(charges)):
            totPot += ((8.98e9)*(charges[n]))/(distOfUnit*dist([posX[n],posY[n],posZ[n]],[x,y,z]))
        self.val = totPot
        self.eqLine = -1

# SCALE - Defines physical length of the axes. 
scale = input("What scale factor would you like for the axes? (Leave blank for the default, 1, which is 1.4 nm.)")
if scale == "":
    scale = 1
else:
    scale = int(scale)
axisLength = scale * 1.4e-9

# NUMBER OF CHARGES - Defines the number of charged objects to consider.
numCharges = input("How many charges would you like to model? (Leave blank for the default, which is 1.)")
if numCharges == "":
    numCharges = 1
else:
    numCharges = int(numCharges)

# CHARGE VALUES - Defines the charge values in C for each charged object.
chargesUnscaled = []
charges = []
manualChargeInput = input("(Y/N) Would you like to manually enter the charge value(s), in C? (Enter N for the default, which is all electrons.)").lower()
if manualChargeInput == "yes" or manualChargeInput == "y":
    for n in range(0, numCharges):
        c = input("Please enter the value for charge " + str(n) + ".")
        chargesUnscaled.append(int(c))
else:
    for n in range(0,numCharges):
        chargesUnscaled.append(-1)
for c in chargesUnscaled:
    charges.append(c*1.6e-19)

# CHARGE POSITIONS - Defines the x-y-z coordinate values for each charge.
posX = []
posY = []
posZ = []
distOfUnit = axisLength/100
for n in range(0,len(charges)):
    posStr = input(("Please enter the coordinates for charge " + str(n) + " with charge " + str(charges[n]) + ".")).strip("() ").split(",")
    posX.append(int(posStr[0]))
    posY.append(int(posStr[1]))
    posZ.append(int(posStr[2]))

# EQUIPOTENTIAL DIFFERENCE - Defines the potential difference between each surface.
eqDiff = input("What potential difference would you like there to be between the equipotential surfaces, in V? (Leave blank for it to be automatically chosen.)")
if eqDiff == "":
    eqDiff = scale
else:
    eqDiff = int(eqDiff)
eqDiffArr = []
for n in range(-int(10/(eqDiff/scale)), int(10/(eqDiff/scale))+1):
    eqDiffArr.append(n*eqDiff)

potVals = []
accuracy = 1 # Beware increasing this value much. Even just 2 makes it take significantly longer, but the results are only marginally better at best, so it's not really worth increasing.
its = 0
progress = "\rRunning...⠀[" + " "*25 + "]"
for x in range(int(-100*accuracy),int(100*accuracy)): # This series of nested for loops creates a potVal object for each point and adds it to an array. The bulk of the calculations happen here.
    sys.stdout.write("\r" + progress)
    for y in range(int(-100*accuracy),int(100*accuracy)):
        pVold = potVal(0, 0, 0)
        pVold.val = 0
        for z in range(int(-100*accuracy),int(100*accuracy)):
            its += 1
            pV = potVal(x/accuracy, y/accuracy, z/accuracy)
            for n in eqDiffArr:
                if (n < pV.val and n > pVold.val) or (n > pV.val and n < pVold.val):
                    pV.eqLine = abs(int(n/eqDiff))
                    potVals.append(pV)
            pVold = pV
    if (x + 100*accuracy)%((2*100*accuracy)/25) == 0:
        progress = progress.replace(" ","█",1)
print("\nDone!\nGenerating graph...")

xVals = []
yVals = []
zVals = []
for n in eqDiffArr: # Creates an array of arrays of x, y, and z for each equipotential surface.
    xVals.append([])
    yVals.append([])
    zVals.append([])
for p in potVals: # Adds the coordinates for each point on each surface to the above arrays.
    xVals[p.eqLine].append(p.x)
    yVals[p.eqLine].append(p.y)
    zVals[p.eqLine].append(p.z)

xValsP = []
yValsP = []
zValsP = []
for n in range(0,len(xVals)): # Trims arrays.
    if len(xVals[n]) != 0:
        xValsP.append(xVals[n])
        yValsP.append(yVals[n])
        zValsP.append(zVals[n])

fig = plt.figure()
ax = plt.axes(projection = '3d')
set_axes_equal(ax)
for n in range(0, len(xValsP)): # Each iteration plots a new surface.
    ax.plot_trisurf(xValsP[n], yValsP[n], zValsP[n], edgecolor = 'none')
    # ax.plot(xValsP[n], yValsP[n], zValsP[n], 'o')

Running...⠀[█████████████████████████]
Done!
Generating graph...
